In [1]:
import pandas as pd
from dateutil.parser import parse, ParserError

In [2]:
customers = pd.read_csv("~/Desktop/Github_content/Module_5/Data/03_Library SystemCustomers.csv")
customers

,Customer ID,Customer Name
0,1.0,Jane Doe
1,2.0,John Smith
2,3.0,Dan Reeves
3,NaN,NaN
4,5.0,William Holden
5,6.0,Jaztyn Forest
6,7.0,Jackie Irving
7,8.0,Matthew Stirling
8,9.0,Emory Ted


In [3]:
# Delete empty rows
customers.dropna(inplace=True)
customers

,Customer ID,Customer Name
0,1.0,Jane Doe
1,2.0,John Smith
2,3.0,Dan Reeves
4,5.0,William Holden
5,6.0,Jaztyn Forest
6,7.0,Jackie Irving
7,8.0,Matthew Stirling
8,9.0,Emory Ted


In [ ]:
# Change Customer ID to INT 
customers['Customer ID'] = customers['Customer ID'].astype('Int32')
customers

,Customer ID,Customer Name
0,1,Jane Doe
1,2,John Smith
2,3,Dan Reeves
4,5,William Holden
5,6,Jaztyn Forest
6,7,Jackie Irving
7,8,Matthew Stirling
8,9,Emory Ted


In [30]:
book = pd.read_csv("~/Desktop/Github_content/Module_5/Data/03_Library Systembook.csv")
book.head()

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID
0,1.0,Catcher in the Rye,"""20/02/2023""",25/02/2023,2 weeks,1.0
1,2.0,Lord of the rings the two towers,"""24/03/2023""",21/03/2023,2 weeks,2.0
2,3.0,Lord of the rings the return of the kind,"""29/03/2023""",25/03/2023,2 weeks,3.0
3,4.0,The hobbit,"""02/04/2023""",25/03/2023,2 weeks,4.0
4,5.0,Dune,"""02/04/2023""",25/03/2023,2 weeks,5.0


In [31]:
len(book)

114

In [32]:
# Create an errors df to keep a track of all dropped records
errors = pd.DataFrame(columns=list(book.columns) + ['Error Type', 'Actionable Error'])
errors

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID,Error Type,Actionable Error


In [33]:
def deNaN_dedupe(df, errors_df):
    # Move rows with all NaN values to errors_df
    all_nans_rows = df[df.isna().all(axis=1)]
    all_nans_rows['Error Type'] = 'Missing all data' 
    all_nans_rows['Actionable Error'] = 'No'
    errors_df = pd.concat([errors_df, all_nans_rows], ignore_index=True)
    df.dropna(how='all', inplace=True)

    # Move rows with some NaN values to errors_df
    some_nans_rows = df[df.isna().any(axis=1)]
    some_nans_rows['Error Type'] = 'Missing some data' 
    some_nans_rows['Actionable Error'] = 'Yes'
    errors_df = pd.concat([errors_df, some_nans_rows], ignore_index=True)
    df.dropna(how='any', inplace=True)

    # Move duplicated rows to errors_df
    dupe_rows = df[df.duplicated(keep='first')]
    dupe_rows['Error Type'] = 'Duplicated'
    dupe_rows['Actionable Error'] = 'No'
    errors_df = pd.concat([errors_df, dupe_rows], ignore_index=True)
    df.drop_duplicates(inplace=True)

    return df, errors_df

In [34]:
book, errors = deNaN_dedupe(book,errors)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3356\2350954504.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_nans_rows['Error Type'] = 'Missing all data'
C:\Users\Admin\AppData\Local\Temp\ipykernel_3356\2350954504.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_nans_rows['Actionable Error'] = 'No'
C:\Users\Admin\AppData\Local\Temp\ipykernel_3356\2350954504.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [35]:
errors

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID,Error Type,Actionable Error
0,NaN,NaN,NaN,NaN,NaN,NaN,Missing all data,No
1,NaN,NaN,NaN,NaN,NaN,NaN,Missing all data,No
2,NaN,NaN,NaN,NaN,NaN,NaN,Missing all data,No
3,NaN,NaN,NaN,NaN,NaN,NaN,Missing all data,No
4,NaN,NaN,NaN,NaN,NaN,NaN,Missing all data,No
...,...,...,...,...,...,...,...,...
89,NaN,NaN,NaN,NaN,NaN,NaN,Missing all data,No
90,NaN,NaN,NaN,NaN,NaN,NaN,Missing all data,No
91,NaN,NaN,NaN,NaN,NaN,NaN,Missing all data,No
92,NaN,NaN,NaN,NaN,NaN,NaN,Missing all data,No


In [36]:
# remove quotation marks from 'Book checkout' to enable date functionality
book['Book checkout'] = book['Book checkout'].str.replace('"','', regex=True)
book.head()

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID
0,1.0,Catcher in the Rye,20/02/2023,25/02/2023,2 weeks,1.0
1,2.0,Lord of the rings the two towers,24/03/2023,21/03/2023,2 weeks,2.0
2,3.0,Lord of the rings the return of the kind,29/03/2023,25/03/2023,2 weeks,3.0
3,4.0,The hobbit,02/04/2023,25/03/2023,2 weeks,4.0
4,5.0,Dune,02/04/2023,25/03/2023,2 weeks,5.0


In [37]:
def is_valid_date(date):
    """
    This function parses a date to see whether it is valid

    Note: Dates need to be in string format to be parsed correctly
    """
    if not date:
        return False
    try:
        parse(date)
        return True
    except ParserError:
        return False

In [38]:
for i in ['Book checkout', 'Book Returned']:
    # remove quotation marks from 'Book checkout' to enable date functionality
    book[i] = book[i].str.replace('"','', regex=True)
    book['valid date'] = book[i].apply(lambda x: is_valid_date(x))
    invalid_dates = book[book['valid date'] == False].drop('valid date', axis=1)
    errors = pd.concat([errors, invalid_dates], ignore_index=True)

    book = book[book['valid date']]
    book.drop('valid date', axis=1, inplace=True)
book

C:\Users\Admin\AppData\Local\Temp\ipykernel_3356\1090769196.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  errors = pd.concat([errors, invalid_dates], ignore_index=True)


,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID
0,1.0,Catcher in the Rye,20/02/2023,25/02/2023,2 weeks,1.0
1,2.0,Lord of the rings the two towers,24/03/2023,21/03/2023,2 weeks,2.0
2,3.0,Lord of the rings the return of the kind,29/03/2023,25/03/2023,2 weeks,3.0
3,4.0,The hobbit,02/04/2023,25/03/2023,2 weeks,4.0
4,5.0,Dune,02/04/2023,25/03/2023,2 weeks,5.0
5,6.0,Little Women,02/04/2023,01/05/2023,2 weeks,1.0
6,7.0,IT,10/04/2063,03/04/2023,2 weeks,6.0
7,8.0,Misery,15/04/2023,03/04/2023,2 weeks,7.0
8,9.0,Catch 22,15/04/2023,16/04/2023,2 weeks,7.0
9,10.0,Animal Farm,20/04/2023,24/04/2023,2 weeks,2.0


In [39]:
# Convert date columns to datetime
book['Book checkout'] = pd.to_datetime(book['Book checkout'], dayfirst=True)
book['Book Returned'] = pd.to_datetime(book['Book Returned'], dayfirst=True)

In [40]:
# Change IDs to int
book['Customer ID'] = book['Customer ID'].astype('Int32')
book['Id'] = book['Id'].astype('Int32')

In [41]:
def loan_time(df, date1, date2):
    df['Loan Days'] = (df[date2] - df[date1]).dt.days

In [42]:
loan_time(book,'Book checkout','Book Returned')

In [46]:
book['valid loan time'] = book['Loan Days'] >= 0
invalid_loan = (book[book['valid loan time'] == False]).drop('valid loan time', axis=1)
errors = pd.concat([errors, invalid_loan], ignore_index=True)

book = book[book['valid loan time']].drop('valid loan time', axis=1)
book

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID,Loan Days
0,1,Catcher in the Rye,2023-02-20,2023-02-25,2 weeks,1,5
5,6,Little Women,2023-04-02,2023-05-01,2 weeks,1,29
8,9,Catch 22,2023-04-15,2023-04-16,2 weeks,7,1
9,10,Animal Farm,2023-04-20,2023-04-24,2 weeks,2,4
10,11,1984,2023-04-23,2023-04-27,2 weeks,8,4
11,12,Little Women,2023-04-02,2023-05-01,2 weeks,1,29
12,13,East of Eden,2023-04-30,2023-05-05,2 weeks,2,5
13,14,America Is in the Heart,2023-05-01,2023-05-07,2 weeks,3,6
14,15,Wuthering Heights,2023-05-01,2023-05-10,2 weeks,9,9
15,16,Dark Tales,2023-05-15,2023-06-01,2 weeks,2,17


In [47]:
invalid_loan

,Id,Books,Book checkout,Book Returned,Days allowed to borrow,Customer ID,Loan Days


In [240]:
from sqlalchemy import create_engine

In [249]:
driver_name = 'SQL Server'
server_name = 'localhost'
database_name = 'LibraryDB'

In [250]:
connection_str = f'mssql+pyodbc://@{server_name}/{database_name}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'

In [251]:
engine = create_engine(connection_str)

In [253]:
book.to_sql('books_bronze', con=engine, if_exists='replace', index=False)

13

In [27]:
def enrichDates(df, date1, date2, errors_df):
    df['Loan Days'] = (df[date2] - df[date1]).dt.days

    df['Valid Loan'] = df['Loan Days'] >= 0
    invalid_loan = df[df['Valid Loan'] == False].drop('Valid Loan', axis=1)
    invalid_loan['Error Type'] = 'Invalid loan'
    invalid_loan['Actionable Error'] = 'Yes'
    errors_df = pd.concat([errors_df, invalid_loan], ignore_index=True)

    df = df[df['Valid Loan']].drop('Valid Loan', axis=1)

    return df, errors_df

In [28]:
enrichDates(book, 'Book checkout', 'Book Returned', errors)

TypeError: unsupported operand type(s) for -: 'str' and 'str'